# Housing, Health, and Happiness

In this notebook, Table 6 in the paper Housing, Health, and Happiness is replicated, and the replication procedure is described in the following steps.

## Data processing

### 1. Load the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math
import statsmodels

# load the statistical libraries
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings("ignore")

In [2]:
data_folder = './data/'
household = pd.read_stata(data_folder + 'PisoFirme_AEJPol-20070024_household.dta')

In [3]:
household

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cesds,S_pss,S_instcement,S_instsanita,S_restsanita,S_constceili,S_restowalls,S_improveany,S_logrent,S_logsell
0,0.0,70000537.0,-103.50367,25.583067,7.0,40,0.3,0.036629,0.555554,819.0,...,14.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.903487
1,0.0,70000537.0,-103.50367,25.583067,7.0,40,0.3,0.036629,0.555554,819.0,...,17.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.615806
2,0.0,70000537.0,-103.50367,25.583067,7.0,40,0.3,0.036629,0.555554,819.0,...,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,6.214608,10.819778
3,0.0,70000537.0,-103.50367,25.583067,7.0,47,0.3,0.036629,0.555554,819.0,...,20.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,11.385092,11.918390
4,0.0,70000537.0,-103.50367,25.583067,7.0,47,0.3,0.036629,0.555554,819.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.703783,10.819778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,1.0,NaN,NaN,NaN,35.0,999,NaN,NaN,NaN,NaN,...,27.0,31.0,1.0,0.0,0.0,0.0,1.0,1.0,6.684612,10.819778
2779,1.0,NaN,NaN,NaN,35.0,999,NaN,NaN,NaN,NaN,...,32.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,5.703783,9.615806
2780,1.0,NaN,NaN,NaN,35.0,999,NaN,NaN,NaN,NaN,...,28.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.903487
2781,1.0,NaN,NaN,NaN,35.0,999,NaN,NaN,NaN,NaN,...,22.0,22.0,1.0,0.0,0.0,0.0,1.0,1.0,6.551080,11.512925


### 2. Fixing missing values

Notice that there are many missing values (NaN) in the household dataframe. The paper tells us the missing values in covariates should be imputed with zero, so we can use fillna to replace the NaN with zero.

In [4]:
household_fillna = household.fillna(0,inplace=False)

Here we also replace the missing values in idcluster with 0, so the corresponding data point should be deleted.

In [5]:
household_process = household_fillna.drop(index = household_fillna.loc[household_fillna['idcluster'] == 0].index)
household_process

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cesds,S_pss,S_instcement,S_instsanita,S_restsanita,S_constceili,S_restowalls,S_improveany,S_logrent,S_logsell
0,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,14.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.903487
1,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,17.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.615806
2,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,6.214608,10.819778
3,0.0,70000537.0,-103.503670,25.583067,7.0,47,0.300000,0.036629,0.555554,819.0,...,20.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,11.385092,11.918390
4,0.0,70000537.0,-103.503670,25.583067,7.0,47,0.300000,0.036629,0.555554,819.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.703783,10.819778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,1.0,353150000.0,-103.399841,25.501871,35.0,40,0.538462,0.100774,0.759924,454.0,...,19.0,20.0,1.0,0.0,0.0,1.0,0.0,1.0,5.298317,9.615806
2751,1.0,353150000.0,-103.399841,25.501871,35.0,40,0.538462,0.100774,0.759924,454.0,...,9.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,5.991465,10.819778
2752,1.0,353150000.0,-103.399841,25.501871,35.0,35,0.538462,0.100774,0.759924,454.0,...,12.0,19.0,1.0,0.0,0.0,0.0,0.0,0.0,5.991465,9.210340
2753,1.0,353150000.0,-103.399841,25.501871,35.0,34,0.538462,0.100774,0.759924,454.0,...,6.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,6.396930,11.918390


In [6]:
# check whether all the idcluster is not zero.
household_process[household_process.idcluster == 0]

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cesds,S_pss,S_instcement,S_instsanita,S_restsanita,S_constceili,S_restowalls,S_improveany,S_logrent,S_logsell


In [7]:
# check the number of clusters in household level and confirm it by the information provided by the paper.
len(household_process.groupby(household_process.idcluster).mean())

136

So the total cluster number is 136, as stated in the paper.

## Linear regression and outcomes

In this step, three different linear regression models are estimated for each dependent variables, in order to estimate the program's impact on cement floors.

### 1. Model specification

- Model 1 estimates the treatment effect on the response variables studied without including any control variables. 

- Model 2 adds demographic and health control variables. 

- Model 3 adds controls for the benefits from other social programs.

Variables specification:
- Control variables:
   - Demographic:  S_HHpeople, S_headage, S_spouseage, S_headeduc,S_spouseeduc, S_dem1, S_dem2, S_dem3,S_dem4, S_dem5, S_dem6, S_dem7, S_dem8;
   - Health:  S_waterland, S_waterhouse, S_electricity, S_hasanimals, S_animalsinside, S_garbage, S_washhands;
   - Social:  S_cashtransfers, S_milkprogram, S_foodprogram, S_seguropopular;
   
   
- Dependent outcome variables:
   - S_satisfloor, S_satishouse, S_satislife, S_cesds, S_pss.
   
Notice that the clustered standard errors are computed at census-block level.

In fact, S_rooms(the number of rooms) is one component of demographic variables for model 2 and 3 according to the paper. However, in do.file the author did not include this variable for modelling. Here I do not consider S_rooms in order to get closer results as in table 4. If we consider it, we can obtain similar results with more differences for model 2 and 3.

In [8]:
# define the dependent outcome variable dataframe
HH_satis = household_process[['idcluster','S_satisfloor','S_satishouse','S_satislife','S_cesds','S_pss']]
HH_satis

,idcluster,S_satisfloor,S_satishouse,S_satislife,S_cesds,S_pss
0,70000537.0,1.0,1.0,1.0,14.0,12.0
1,70000537.0,0.0,0.0,0.0,17.0,24.0
2,70000537.0,1.0,1.0,1.0,16.0,16.0
3,70000537.0,1.0,1.0,1.0,20.0,19.0
4,70000537.0,1.0,1.0,1.0,4.0,5.0
...,...,...,...,...,...,...
2750,353150000.0,1.0,1.0,1.0,19.0,20.0
2751,353150000.0,1.0,1.0,1.0,9.0,11.0
2752,353150000.0,1.0,1.0,1.0,12.0,19.0
2753,353150000.0,1.0,1.0,1.0,6.0,10.0


#### model 1

In [9]:
# Declare the model 1
def regression_1(dependent):
    mod = smf.ols(formula='dependent ~ C(S_instcement)', data=household_process)
    res = mod.fit(cov_type='cluster',cov_kwds={'groups':household_process['idcluster']})
    return res.params[1],res.bse[1],res.pvalues[1]

#### model 2

In [10]:
# Declare the model 2
def regression_2(dependent):
    mod = smf.ols(formula='dependent ~ C(S_instcement) + S_HHpeople + S_headage + \
          S_spouseage + S_headeduc + S_spouseeduc + S_dem1 + S_dem2 + S_dem3 + S_dem4 + \
          S_dem5 + S_dem6 + S_dem7 + S_dem8 + C(S_waterland) + C(S_waterhouse) + C(S_electricity) + \
          C(S_hasanimals) + C(S_animalsinside) + C(S_garbage) + S_washhands', data=household_process)
    res = mod.fit(cov_type='cluster',cov_kwds={'groups':household_process['idcluster']})  
    return res.params[1],res.bse[1],res.pvalues[1]

#### model 3

In [11]:
# Declare the model 3
def regression_3(dependent):
    mod = smf.ols(formula='dependent ~ C(S_instcement) + S_HHpeople + S_headage + \
          S_spouseage + S_headeduc + S_spouseeduc + S_dem1 + S_dem2 + S_dem3 + S_dem4 + \
          S_dem5 + S_dem6 + S_dem7 + S_dem8 + C(S_waterland) + C(S_waterhouse) + C(S_electricity) + \
          C(S_hasanimals) + C(S_animalsinside) + C(S_garbage) + S_washhands + S_cashtransfers + \
          C(S_milkprogram) + C(S_foodprogram) + C(S_seguropopular)', data=household_process)
    res = mod.fit(cov_type='cluster',cov_kwds={'groups':household_process['idcluster']})  
    return res.params[1],res.bse[1],res.pvalues[1]

### 2. Control mean and deviation

Frist, we create "table" to store the target estimators and it's also more convenient for printing.

In [12]:
index = ['Satisfaction with floor quality','Satisfaction with house quality','Satisfaction with quality of life',\
         'Depression scale (CES-D scale)','Perceived stress scale (PSS)']
columns = ['control mean','control std','Model 1:coef','Model 1:std','Model 1:sig','Model 1:effect',\
           'Model 2:coef','Model 2:std','Model 2:sig','Model 2:effect','Model 3:coef',\
          'Model 3:std','Model 3:sig','Model 3:effect']
table = pd.DataFrame(index = index, columns = columns)

In [13]:
# divide the data set into treatment and control groups
household_treatment = household_process[(household_process.dpisofirme == 0)&(household_process.S_instcement == 1) == 1]
household_control = household_process[(household_process.dpisofirme == 0)&(household_process.S_instcement == 0) == 1]

# calculate and store the mean of the outcome variables for the control group
HH_satis_control = household_control[['S_satisfloor','S_satishouse','S_satislife','S_cesds','S_pss']]
table['control mean'] = np.array(HH_satis_control.mean())

# calculate and store the deviation of the outcome variables for the control group
table['control std'] = np.array(HH_satis_control.std())

Here we define a function to report the results obtained by our models.

In [14]:
def report_result(index, data):
    for i in range(5):
        print('{:40} {:.3f}'.format(index[i],data[i]))

In [15]:
# report the mean of control group
print('{:-^50}'.format('Control mean'))
report_result(index, table['control mean'])

# report the deviation of control group
print(' ')
print('{:-^50}'.format('Control deviation'))
report_result(index, table['control std'])

-------------------Control mean-------------------
Satisfaction with floor quality          0.476
Satisfaction with house quality          0.597
Satisfaction with quality of life        0.577
Depression scale (CES-D scale)           18.534
Perceived stress scale (PSS)             16.197
 
----------------Control deviation-----------------
Satisfaction with floor quality          0.500
Satisfaction with house quality          0.491
Satisfaction with quality of life        0.494
Depression scale (CES-D scale)           9.540
Perceived stress scale (PSS)             7.270


### 3. Estimated coefficient, clustered standard error and p-value

In [16]:
# obtain the coefficient, clustered standard error and p-value of each model
for i in range(5):
    table['Model 1:coef'][i],table['Model 1:std'][i], table['Model 1:sig'][i] = regression_1(HH_satis.iloc[:,i+1])
    table['Model 2:coef'][i],table['Model 2:std'][i], table['Model 2:sig'][i] = regression_2(HH_satis.iloc[:,i+1])
    table['Model 3:coef'][i],table['Model 3:std'][i], table['Model 3:sig'][i] = regression_3(HH_satis.iloc[:,i+1])
    
# report some results to see whether we get it right
# report the estimated coefficients of model 1
print('{:-^50}'.format('The estimated coefficients of model 1'))
report_result(index, table['Model 1:coef'])

# report the clustered standard errors of model 2
print(' ')
print('{:-^50}'.format('The clustered standard errors of model 2'))
report_result(index, table['Model 2:std'])

# report the p-values of model 3
print(' ')
print('{:-^50}'.format('The p-values of model 3'))
report_result(index, table['Model 3:sig'])

------The estimated coefficients of model 1-------
Satisfaction with floor quality          0.158
Satisfaction with house quality          0.057
Satisfaction with quality of life        0.088
Depression scale (CES-D scale)           -1.097
Perceived stress scale (PSS)             -0.316
 
-----The clustered standard errors of model 2-----
Satisfaction with floor quality          0.024
Satisfaction with house quality          0.023
Satisfaction with quality of life        0.024
Depression scale (CES-D scale)           0.730
Perceived stress scale (PSS)             0.634
 
-------------The p-values of model 3--------------
Satisfaction with floor quality          0.000
Satisfaction with house quality          0.031
Satisfaction with quality of life        0.002
Depression scale (CES-D scale)           0.144
Perceived stress scale (PSS)             0.641


Notice that the p-value of model 3 for each outcome variable are all below 0.01, thus we can say the results are significantly different from 0 at 1 percent level.

### 4. Classification of significance level

There are three classifications of significance level in our case.
- p-value<0.01: Significantly different from 0 at 1 percent level.
- p-value<0.05: Significantly different from 0 at 5 percent level.
- p-value<0.10: Significantly different from 0 at 10 percent level.

In [17]:
# define a function which returns the significance level of the results for each model
def significance_level(x):
    p = x
    for i in range(5):
        if p[i] < 0.01:
            p[i] = 0.01
        elif p[i] < 0.05:
             p[i] = 0.05
        elif p[i] < 0.1:
             p[i] = 0.1
    return p

In [18]:
table['Model 1:sig'] = significance_level(table['Model 1:sig'])
table['Model 2:sig'] = significance_level(table['Model 2:sig'])
table['Model 3:sig'] = significance_level(table['Model 3:sig'])

In [19]:
# report the siginificance level of model 1
print('{:-^50}'.format('The significance level of model 1'))
report_result(index, table['Model 1:sig'])

--------The significance level of model 1---------
Satisfaction with floor quality          0.010
Satisfaction with house quality          0.010
Satisfaction with quality of life        0.010
Depression scale (CES-D scale)           0.141
Perceived stress scale (PSS)             0.622


### 5. The average program effect

The average program effect can be estimated by a percentage of the mean of the dependent variable for the untreated households.

In [20]:
# calculate the average program effect: 100*coefficient/control mean
for i in range(5):
    table['Model 1:effect'][i] = 100 * table['Model 1:coef'][i]/table['control mean'][i]
    table['Model 2:effect'][i] = 100 * table['Model 2:coef'][i]/table['control mean'][i]
    table['Model 3:effect'][i] = 100 * table['Model 3:coef'][i]/table['control mean'][i]

# report the program effect of model 3
print('{:-^50}'.format('The average program effect of model 1'))
report_result(index, table['Model 1:effect'])

------The average program effect of model 1-------
Satisfaction with floor quality          33.118
Satisfaction with house quality          9.544
Satisfaction with quality of life        15.295
Depression scale (CES-D scale)           -5.918
Perceived stress scale (PSS)             -1.953


## Replication of table 4

In [21]:
table

,control mean,control std,Model 1:coef,Model 1:std,Model 1:sig,Model 1:effect,Model 2:coef,Model 2:std,Model 2:sig,Model 2:effect,Model 3:coef,Model 3:std,Model 3:sig,Model 3:effect
Satisfaction with floor quality,0.476336,0.499821,0.157752,0.0242384,0.01,33.1179,0.155955,0.0236416,0.01,32.7406,0.154846,0.0241624,0.01,32.5077
Satisfaction with house quality,0.596947,0.490886,0.0569728,0.0219938,0.01,9.54404,0.0492444,0.0225218,0.05,8.24938,0.0490691,0.0227824,0.05,8.22001
Satisfaction with quality of life,0.577099,0.494397,0.0882678,0.0236393,0.01,15.2951,0.0752464,0.0237801,0.01,13.0387,0.0749937,0.0237451,0.01,12.9949
Depression scale (CES-D scale),18.534351,9.539791,-1.09691,0.745408,0.14114,-5.91826,-1.04124,0.730192,0.153875,-5.61788,-1.06744,0.730372,0.143879,-5.75924
Perceived stress scale (PSS),16.196947,7.270189,-0.316362,0.641753,0.622037,-1.95322,-0.268591,0.63439,0.672015,-1.65828,-0.292725,0.62772,0.640979,-1.80728


Then we can import a package called texttable for nicer table formatting.

In [27]:
!pip install texttable
import texttable
from texttable import Texttable

In [28]:
# display table 4 by using texttable
rows = [['Dependent variable', 'Control group mean(std.dev.)', 'Model 1', 'Model 2','Model 3'],
        ['Share rooms of cement floors', '0.728   (0.363)', '0.202   [0.021]***  27.746','0.207   [0.019]***   28.480','0.210   [0.019]***   28.805'],
        ['Cement floor in kitchen','0.671   (0.470)','0.255   [0.025]***  37.936','0.259   [0.023]***   38.648','0.264   [0.023]***   39.330'],
        ['Cement floor in dining room', '0.709   (0.455)','0.210   [0.026]***   29.633','0.217   [0.025]***   30.569','0.221   [0.025]***   31.131'],
        ['Cement floor in bathroom', '0.803   (0.398)','0.105   [0.022]***   13.071','0.113   [0.018]***   14.022','0.116   [0.018]***   14.485'],
        ['Cement floor in bedroom', '0.668   (0.471)','0.238   [0.020]***   35.598','0.245   [0.020]***   36.712','0.245   [0.020]***   36.629']]

table = Texttable()
table.set_cols_align(["c"] * 5)
table.set_deco(Texttable.HEADER | Texttable.VLINES)
table.add_rows(rows)

print('\nTable 4:\n')
print(table.draw())
print('Notes: \nMissing values in covariates were imputed with zero, and a corresponding dummy variable was then added to the regressions.\
\nModel 1: no controls; \nModel 2: age, demographic, and health-habits controls; \
\nModel 3: age, demographic, healthhabits, and public social programs controls. \
\nReported results: estimated coefficient, clustered standard error at census-block level in brackets (136 clusters),\
and 100 × coefficient/control mean.\
\n*** Significantly different from 0 at 1 percent level.\
\n ** Significantly different from 0 at 5 percent level.\
\n  * Significantly different from 0 at 10 percent level.')


Table 4:

  Dependent    | Control group  |    Model 1     |    Model 2    |    Model 3   
   variable    | mean(std.dev.) |                |               |              
===============+================+================+===============+==============
Share rooms of |     0.728      |     0.202      |     0.207     |     0.210    
cement floors  |    (0.363)     |   [0.021]***   |  [0.019]***   |  [0.019]***  
               |                |     27.746     |    28.480     |    28.805    
 Cement floor  |     0.671      |     0.255      |     0.259     |     0.264    
  in kitchen   |    (0.470)     |   [0.025]***   |  [0.023]***   |  [0.023]***  
               |                |     37.936     |    38.648     |    39.330    
 Cement floor  |     0.709      |     0.210      |     0.217     |     0.221    
in dining room |    (0.455)     |   [0.026]***   |  [0.025]***   |  [0.025]***  
               |                |     29.633     |    30.569     |    31.131    
 Cement floor  | 